In [57]:
import time
import os

import numpy as np
import cv2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pyautogui

In [58]:
def click_and_write_current_screen(img_background, gry_background, template, sentence, result_file_path):
    location = template_match(gry_background, template)
    draw_rectangle(img_background, template, location)
    click_location = get_click_location(template, location)
    cv2.imwrite(result_file_path, img_background)
    pyautogui.click(click_location)
    pyautogui.typewrite(sentence)
    
def click_current_screen(img_background, gry_background, template, result_file_path):
    location = template_match(gry_background, template)
    draw_rectangle(img_background, template, location)
    click_location = get_click_location(template, location)
    cv2.imwrite(result_file_path, img_background)
    pyautogui.click(click_location)
    
def template_match(img_grey, template):
    res = cv2.matchTemplate(img_grey, template, cv2.TM_CCOEFF_NORMED)
    threshold = get_threshold(res)
    result = np.where(res > threshold)
    result = zip(*result[::-1])
    return list(result)[0]

def get_click_location(template, res):
    w, h = template.shape[::-1]
    return (res[0] + w / 2, res[1] + h / 2)

def draw_rectangle(original, template, res):
    w, h = template.shape[::-1]
    cv2.rectangle(original, res, (res[0] + w, res[1] + h), (0,0,255), 2)

def get_threshold(nd_array):
    threshold = 0.6
    while not select_specific_position(nd_array, threshold):
        print(threshold)
        threshold += 0.02
    
    return threshold

def select_specific_position(nd_array, threshold):
    tmp_array = np.where(nd_array > threshold)
    if len(tmp_array) == 2:
        return True
    else:
        return False

In [59]:
certificate_password = os.getenv("CERTIFICATE_PASSWORD")
driver_path = os.getenv("DRIVER_PATH")
dert_path = os.getenv('DERT_PATH')
key_path = os.getenv('KEY_PATH')

### 홈택스 페이지접근


In [60]:
options = Options()
# chrome user agent를 지정해주기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36")


driver = webdriver.Chrome(driver_path, options=options)

driver.get('https://www.hometax.go.kr/websquare/websquare.wq?w2xPath=/ui/pp/index.xml&tmIdx=0&tm2lIdx=&tm3lIdx=')

cookies = driver.get_cookies()

button_for_login = driver.find_element_by_xpath("//a[@id='group1544'][@class='w2group ']")
time.sleep(5)
driver.save_screenshot('hometax/current_page.png')

True

### 홈택스 로그인페이지 

In [61]:
button_for_login.click()
time.sleep(5)
driver.maximize_window()
driver.switch_to.frame('txppIframe')
driver.save_screenshot('hometax/current_page.png')

True

### 홈택스 공인인증서 창 클릭 

In [ ]:
button_for_login = driver.find_element_by_xpath("//input[@type='button'][@id='trigger38'][@value='공인인증서 로그인']")
time.sleep(5)
button_for_login.click()
driver.save_screenshot('hometax/current_page.png')

### 인증서 로그인을 위한 클릭

In [ ]:
time.sleep(5)
driver.switch_to.default_content()
driver.switch_to.frame('txppIframe')
driver.switch_to.frame("dscert")
button_for_searching_certificate = driver.find_element_by_xpath("//span[@id='get_cert']")
button_for_searching_certificate.click()
driver.save_screenshot('hometax/current_page.png')

### 인증서 찾기

In [ ]:
time.sleep(3)
input_for_cert_path = driver.find_element_by_xpath("//input[@type='file'][@id='filefile2']")
input_for_cert_path.click()
driver.save_screenshot('hometax/current_page.png')

In [ ]:
time.sleep(3)
pyautogui.screenshot('hometax/current.png')

In [ ]:
img_file_dialog = cv2.imread('hometax/current.png')
grey_file_dialog = cv2.cvtColor(img_file_dialog, cv2.COLOR_BGR2GRAY)
template_for_filedialog = cv2.imread('hometax/cert.png', 0)
template_for_button = cv2.imread('hometax/button.png', 0)

### FILE PATH 세팅 

In [ ]:
file_path = '\"' + dert_path + '\"  \"' + key_path + '\"'

In [ ]:
time.sleep(3)
click_and_write_current_screen(img_file_dialog, grey_file_dialog, template_for_filedialog, file_path, result_file_path='hometax/res.png')
click_current_screen(img_file_dialog, grey_file_dialog, template_for_button, result_file_path='hometax/res.png')

### file path 및 비밀번호 입력

In [ ]:
input_for_password = driver.find_element_by_xpath("//input[@type='password'][@id='add_browser_password']")
input_for_password.send_keys(certificate_password)
time.sleep(3)
button_for_login = driver.find_element_by_xpath("//button[@type='button'][@id='btn_common_confirm']")
button_for_login.click()
driver.save_screenshot('hometax/current_page.png')